In [39]:
import pandas as pd
from collections import defaultdict

In [45]:
ranks = pd.read_csv("data/ranks.csv")
ranks

,year,team,rank
0,2003,ARG,35
1,2003,AUS,15
2,2003,BRA,6
3,2003,CAN,12
4,2003,CHN,4
...,...,...,...
123,2023,SUI,21
124,2023,SWE,2
125,2023,USA,1
126,2023,VIE,34


In [46]:
def get_quartile(row):
    subset = ranks[ranks["year"] == row["year"]]
    quartiles = subset["rank"].quantile([0.25, 0.5, 0.75])
    if row["rank"] < quartiles[0.25]:
        return 1
    elif row["rank"] < quartiles[0.5]:
        return 2
    elif row["rank"] < quartiles[0.75]:
        return 3
    else:
        return 4

ranks["quartile"] = ranks.apply(get_quartile, axis=1)
ranks

,year,team,rank,quartile
0,2003,ARG,35,4
1,2003,AUS,15,3
2,2003,BRA,6,2
3,2003,CAN,12,3
4,2003,CHN,4,1
...,...,...,...,...
123,2023,SUI,21,3
124,2023,SWE,2,1
125,2023,USA,1,1
126,2023,VIE,34,3


In [49]:
year = "2023"
df = pd.read_csv("data/matches_" + year + ".csv")
df = df[df["stage"] == "First Stage"]
year_ranks = ranks[ranks["year"] == int(year)][["team", "quartile"]].set_index("team")
df = df.set_index("home").join(year_ranks).rename(
    columns={"quartile": "home_quartile"}
).reset_index(drop=True).set_index("away").join(year_ranks).rename(
    columns={"quartile": "away_quartile"}
).reset_index(
    drop=True
)

# records[q1][q2] = (W, L, D)
records = defaultdict(lambda: defaultdict(lambda: [0, 0, 0]))
for i, row in df.iterrows():
    if row["home_score"] > row["away_score"]:
        records[row["home_quartile"]][row["away_quartile"]][0] += 1
        records[row["away_quartile"]][row["home_quartile"]][1] += 1
    elif row["home_score"] < row["away_score"]:
        records[row["home_quartile"]][row["away_quartile"]][1] += 1
        records[row["away_quartile"]][row["home_quartile"]][0] += 1
    else:
        records[row["home_quartile"]][row["away_quartile"]][2] += 1
        records[row["away_quartile"]][row["home_quartile"]][2] += 1
        
records

defaultdict(<function __main__.<lambda>()>,
            {2: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {3: [3, 2, 1],
                          1: [2, 4, 1],
                          4: [5, 3, 1],
                          2: [1, 1, 0]}),
             3: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: [2, 3, 1],
                          1: [1, 6, 1],
                          4: [1, 2, 3],
                          3: [1, 1, 2]}),
             1: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: [4, 2, 1],
                          4: [5, 0, 2],
                          3: [6, 1, 1],
                          1: [0, 0, 2]}),
             4: defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
                         {2: [3, 5, 1],
                          1: [0, 5, 2],
                          3: [2, 1, 3],
                          4: [1, 1, 0]